In [ ]:
## Import dependencies
import sys, os
import pandas as pd


In [ ]:
# Step 1: Load the CSV file
file_path = 'data\\20241029_LCME.csv'  # Replace with your CSV file path
lcme_df = pd.read_csv(file_path)
lcme_df.head()

In [ ]:
# Step 2: Remove rows where the department is NaN or empty
lcme_df_2 = lcme_df[lcme_df['Department'].notna() & (lcme_df['Department'] != '')]

# Check for any remaining NaN values in the Department column
print("Number of NaN values in 'Department':", lcme_df_2['Department'].isna().sum())

# Check for any remaining empty strings in the Department column
print("Number of empty strings in 'Department':", (lcme_df_2['Department'] == '').sum())

# Display unique values in the Department column to confirm no blanks
print("Unique values in 'Department':", lcme_df_2['Department'].unique())

# Check the shape before and after removing rows
print("Original DataFrame shape:", lcme_df.shape)
print("Filtered DataFrame shape:", lcme_df_2.shape)

lcme_df_2.head()

In [ ]:
# Step 3: Remove trailing semicolons from department names
lcme_df_2['Department'] = lcme_df_2['Department'].str.rstrip(';')

# Check for any departments with a trailing semicolon
print("Departments with trailing semicolons:", lcme_df_2[lcme_df_2['Department'].str.endswith(';')])

# Display unique department names to confirm no trailing semicolons
print("Unique department names:", lcme_df_2['Department'].unique())

lcme_df_2.head()


In [ ]:
# Step 4: Replace instances of different school names with 'Feinberg School of Medicine'
school_names_to_replace = [' Developmental Biology; Feinberg School of Medicine',
                            'Developmental Biology; Feinberg School of Medicine',
                            'Office for Research', 
                            'ISGMH - Institute for Sexual and Gender Minority Health and Wellbeing', 
                            'Office for Research Safety', 
                            ' Judd A. and Marjorie Weinberg College of Arts and Sciences',
                            'Judd A. and Marjorie Weinberg College of Arts and Sciences',
                            'DevSci - Institute for Innovations in Developmental Sciences', 
                            'Office for Research Information Technology',
                            'NU Clinical and Translational Sciences Institute', 
                            'Office for Research', 
                            'Office of Finance and Administration', 
                            ]  # Add any variations here

lcme_df_2['School'] = lcme_df_2['School'].str.strip().replace(school_names_to_replace, 'Feinberg School of Medicine')

# Display unique department names
print("Unique department names after replacement:", lcme_df_2['School'].unique())

lcme_df_2.head()

In [ ]:
# Step 3: Define the department mapping
department_mapping = {
    'Cell & Developmental Biology': ['Cell &amp'],
    'Medical Education': ['Medical Education, Physician Assistant Program',
                          'Medical Scientist Training Program',
                          ],
    'Medicine': ['Medicine, Allergy Division', 
                            'Medicine, Cardiology Division', 
                            'Medicine, Endocrinology Division', 
                            'Medicine, Gastroenterology Division', 
                            'Medicine, General Medicine Division', 
                            'Medicine, Geriatrics Division', 
                            'Medicine, Hematology Oncology Division', 
                            'Medicine, Hepatology Division', 
                            'Medicine, Hospital Medicine Division', 
                            'Medicine, Infectious Diseases Division', 
                            'Medicine, Nephrology Division', 
                            'Medicine, Pulmonary Division', 
                            'Medicine, Rheumatology Division' 
                            ],
    'Neurology': ['Mesulam Center for Cognitive Neurology and Alzheimers Disease'
                    'Neuroscience',
                    'Northwestern University Interdepartmental Neurosciences' 
                    ],
    'Otolaryngology': ['Otolaryngology, Dental Surgery Division'],
    'Pediatrics': ['Pediatrics (Stanley Manne)'],
    'Surgery':['Surgery, Breast Surgery Division', 
                'Surgery, Cardiac Surgery Division', 
                'Surgery, Clinical Trials Unit', 
                'Surgery, Gastrointestinal/Endo Division', 
                'Surgery, Plastic Surgery Division', 
                'Surgery, Surgical Oncology Division', 
                'Surgery, Thoracic Surgery Division', 
                'Surgery, Transplant Surgery Division', 
                'Surgery, Trauma and Critical Care Division', 
                'Surgery, Vascular Surgery Division'
                  ]
    
    # Add more mappings as needed
}

# Step 4: Create a reverse mapping for the replace function
replace_dict = {old_name: new_name for new_name, old_names in department_mapping.items() for old_name in old_names}

# Step 5: Replace department names
lcme_df_2['Department'] = lcme_df_2['Department'].replace(replace_dict)

## Print unique department names after replacement
print("Unique department names after replacement:", lcme_df_2['Department'].unique())

lcme_df_2.head()

In [ ]:
# Step 6: Define departments to drop
departments_to_drop = ['Buehler Center for Health Policy and Economics', 
                        'Center for Education in Health Sciences', 
                        'Center for Education in Medicine', 
                        'Center for Food Allergy and Asthma Research', 
                        'Center for Genetic Medicine',
                        'Faculty Affairs', 
                        'Feinberg Cardiovascular Research Institute', 
                        'Galter Health Science Library', 
                        'Global Health Institute', 
                        'Integrated Grad Program', 
                        'Lurie Cancer Center',
                        'NU Clinical and Translational Sciences Institute',
                        'Office of Finance and Administration',
                       'Office for Research'

                      ]  # Replace with the names of departments to drop

# Step 7: Filter out entries from the specified departments
lcme_df_3 = lcme_df_2[~lcme_df_2['Department'].isin(departments_to_drop)]

## Print unique department names after replacement
print("Unique department names after replacement:", lcme_df_3['Department'].unique())

duplicate_count = lcme_df_3.duplicated(subset=['Department', 'Publication ID']).sum()
print(f"Number of duplicate entries in lcme_df_3: {duplicate_count}")


lcme_df_3.head()

In [ ]:
# Step 8: Remove duplicate publications for each department
# Assuming the unique identifier column is named 'unique_id'
lcme_df_4 = lcme_df_3[['Publication ID', 'title', 'Department', 'Type']].drop_duplicates()

print(f"Shape of lcme_df_3: {lcme_df_3.shape}")
print(f"Shape of lcme_df_4: {lcme_df_4.shape}")

# # Show the first few rows of lcme_df_3 and lcme_df_4 for comparison
# print("Sample data from lcme_df_3:")
# print(lcme_df_3[['Department', 'Publication ID']].head(10))

# print("\nSample data from lcme_df_4:")
# print(lcme_df_4[['Department', 'Publication ID']].head(10))

print("Unique publication IDs in lcme_df_4:", lcme_df_4['Publication ID'].nunique())
print("Unique departments in lcme_df_4:", lcme_df_4['Department'].nunique())


lcme_df_4.head(20)

In [ ]:
# Step 7: Count unique document types by department
# Create a pivot table to count the unique document types
lcme_df_5 = lcme_df_4.pivot_table(index='Department', 
                                               columns='Type', 
                                               values='Publication ID', 
                                               aggfunc='nunique', 
                                               fill_value=0)


print(f"Original unique publication count: {original_count}")
print(f"New unique publication count after removing duplicates: {new_count}")

# Display the result
lcme_df_5.head(20)

In [ ]:
# Step 9: Make any final edits

# Drop the 'Book' and 'Chapter' columns
lcme_df_6 = lcme_df_5.drop(columns=['Conference', 'Internet publication', 'Other', 'Poster', 'Preprint', 'Report'])


## Create a new column for merged counts
lcme_df_6['Published Books/Book Chapters'] = lcme_df_6['Book'] + lcme_df_6['Chapter']

# Drop the 'Book' and 'Chapter' columns
lcme_df_7 = lcme_df_6.drop(columns=['Book', 'Chapter'])

# Rename the "Journal Article" column
lcme_df_8 = lcme_df_7.rename(columns={'Journal article': 'Articles in Peer-reviewed Journals'}) 

lcme_df_8.head(20)

In [ ]:
# Step 10: Export the pivot table to an Excel file

output_file_path = 'output/department_document_counts.xlsx'  # Define your output file name
lcme_df_8.to_excel(output_file_path, sheet_name='Counts')
